In [ ]:
!pip install opencv-python


In [23]:
!pip install opencv-python


In [1]:
!pip install --upgrade numpy

In [2]:
import cv2

In [3]:
import numpy as np

In [10]:
%%cmd
pip install cmake

Microsoft Windows [Version 10.0.19045.3448]
(c) Microsoft Corporation. All rights reserved.

C:\anaconda\envs>pip install cmake
     ---------------------------------------- 34.6/34.6 MB 9.2 MB/s eta 0:00:00

C:\anaconda\envs>

In [4]:
%%cmd
where python

Microsoft Windows [Version 10.0.19045.3448]
(c) Microsoft Corporation. All rights reserved.

C:\anaconda\envs>where python
C:\anaconda\python.exe

C:\anaconda\envs>

In [5]:
import cv2

In [6]:
%%cmd
python --version

Microsoft Windows [Version 10.0.19045.3448]
(c) Microsoft Corporation. All rights reserved.

C:\anaconda\envs>python --version
Python 3.9.13

C:\anaconda\envs>

In [7]:
%%cmd 
pip install "C:\Users\NAMRATA\machine_learning"

Microsoft Windows [Version 10.0.19045.3448]
(c) Microsoft Corporation. All rights reserved.

C:\anaconda\envs>pip install "C:\Users\NAMRATA\machine_learning"

C:\anaconda\envs>

ERROR: Directory 'C:\\Users\\NAMRATA\\machine_learning' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [15]:
%%cmd 
pip install "C:\anaconda\envs"

Microsoft Windows [Version 10.0.19045.3448]
(c) Microsoft Corporation. All rights reserved.

C:\anaconda\envs>pip install "C:\anaconda\envs"

C:\anaconda\envs>

ERROR: Directory 'C:\\anaconda\\envs' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [8]:
%%cmd 
pip install "C:\Users\NAMRATA\Downloads\dlib-19.22.99-cp39-cp39-win_amd64.whl"

Microsoft Windows [Version 10.0.19045.3448]
(c) Microsoft Corporation. All rights reserved.

C:\anaconda\envs>pip install "C:\Users\NAMRATA\Downloads\dlib-19.22.99-cp39-cp39-win_amd64.whl"
Processing c:\users\namrata\downloads\dlib-19.22.99-cp39-cp39-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

C:\anaconda\envs>

In [9]:
import cv2

In [10]:
%%cmd
pip install face-recognition

Microsoft Windows [Version 10.0.19045.3448]
(c) Microsoft Corporation. All rights reserved.

C:\anaconda\envs>pip install face-recognition

C:\anaconda\envs>

In [11]:
!pip install numpy

In [12]:
!pip install pillow

In [16]:
!pip install --user opencv-python-headless

  Using cached opencv_python_headless-4.8.1.78-cp37-abi3-win_amd64.whl (38.0 MB)


In [14]:
import cv2

In [17]:
import cv2
import numpy as np
import face_recognition
import os
import csv
from datetime import datetime, timedelta
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication


# Set the path to the directory containing subdirectories for each student
path = 'goku'
images = []
classNames = []

# Loop through all subdirectories (each representing a student)
for student_dir in os.listdir(path):
    student_path = os.path.join(path, student_dir)
    
    if os.path.isdir(student_path):
        # Loop through all image files within the student's directory
        for image_file in os.listdir(student_path):
            image_path = os.path.join(student_path, image_file)
            
            if os.path.isfile(image_path):
                # Load the image and append it to the 'images' list
                curImg = cv2.imread(image_path)
                
                if curImg is not None:
                    images.append(curImg)
                    classNames.append(student_dir)  # Use the student's directory name as the class name
                else:
                    print(f"Warning: Unable to load image {image_path}")

# Now, 'images' contains all the loaded images, and 'classNames' contains their corresponding student names.


# Function to find encodings of known faces
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img, model="cnn")[0]  # Use the CNN model for face recognition
        encodeList.append(encode)
    return encodeList



# Function to mark attendance
def markAttendance(name, csv_file, alert_message=None):
    if name != "Unknown":  # Only mark attendance if the name is not "Unknown"
        with open(csv_file, 'a', newline='') as f:
            now = datetime.now()
            date = now.strftime('%Y-%m-%d')
            day_of_week = now.strftime('%A')
            time_stamp = now.strftime('%H:%M:%S')
            
            if alert_message:
                row = [name, date, day_of_week, time_stamp, alert_message]
            else:
                row = [name, date, day_of_week, time_stamp, alert_message]

            writer = csv.writer(f)
            writer.writerow(row)




# Function to send an email with the CSV file as an attachment
def send_email(csv_file):
    # Email configuration
    sender_email = 'subhrabandyopadhyay15@gmail.com'  # Change to your email
    sender_password = 'lzks uedv fqco egir'  # Change to your email password
    recipient_email = 'namrata2882m@gmail.com'  # Change to recipient's email
    smtp_server = 'smtp.gmail.com'  # Change to your SMTP server
    smtp_port = 587  # Change to your SMTP server's port

    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = 'Attendance Report'

    # Attach the CSV file
    with open(csv_file, 'rb') as file:
        part = MIMEApplication(file.read(), Name=os.path.basename(csv_file))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(csv_file)}"'
        msg.attach(part)

    # Connect to the SMTP server and send the email
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, sender_password)
        text = msg.as_string()
        server.sendmail(sender_email, recipient_email, text)
        server.quit()
        print(f"Email sent successfully to {recipient_email}")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

# Initialize known face encodings
encodeListKnown = findEncodings(images)
print('Encoding Complete')

# Initialize webcam capture
cap = cv2.VideoCapture(0)

# Generate a unique CSV file name based on the current timestamp
csv_file = f'Attendance_Book_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'

# Define the header row
header = ['  NAME  ','  DATE  ',   '  DAY_OF_WEEK  ', '  TIME-STAMP  ', '  ATTENDANCE-STATUS  ']
# Write the header row to the CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)


# Initialize attendance record
attendance = {}
crypto={}



# Set the threshold time for student presence (adjust as needed)
presence_threshold = timedelta(seconds=10)
absence_threshold = timedelta(seconds=5)
attendance_recorded = set()  # Initialize the set to keep track of recorded attendance

# Set the tolerance for face recognition (adjust as needed, lower values are stricter)
tolerance = 0.5

# Create a set of all class names
all_class_names = set(classNames)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS, model="cnn")  # Use the CNN model for face detection
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    # Create a list to keep track of students present in the current frame
    students_present = []

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace, tolerance=tolerance)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            student_name = classNames[matchIndex].upper()
            confidence = 100 - (faceDis[matchIndex] * 100)  # Calculate confidence percentage
            students_present.append(student_name)

            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, f"{student_name} ({confidence:.2f}%)", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)


            if student_name not in attendance:
                alert_message = f"Alert: Student {student_name} has entered the class!"
                print(alert_message)
                #markAttendance(student_name, csv_file, alert_message)
                attendance[student_name] = datetime.now()
                crypto[student_name]= datetime.now()
                
            else:
                last_seen = attendance[student_name]
                current_time = datetime.now()
                time_difference = current_time - last_seen

                # Check if the student has been present for at least 10 seconds
                if time_difference >= presence_threshold:
                    if student_name not in attendance_recorded:
                        markAttendance(student_name, csv_file, "FINAL ATTENDANCE STATUS: PRESENT")
                        attendance_recorded.add(student_name)
                        
                        print("FINAL ATTENDANCE STATUS PRESENT:", student_name)
                        
        else:
            # Unknown person logic
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red rectangle
            cv2.putText(img, "Unknown", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

           

    # Check for students who were previously marked as present but are not in the current frame
    for student_name, last_seen in list(attendance.items()):
        if student_name not in students_present:
            current_time = datetime.now()
            time_difference = current_time - last_seen
            if time_difference > absence_threshold:
                alert_message = f"Alert: Student {student_name} has left the class!"
                print(alert_message)
                #markAttendance(student_name, csv_file, alert_message)
                
                
                crypto[student_name]= datetime.now()
                del attendance[student_name]
                
                


    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # Before exiting the program, add a final attendance entry for each student
        for student_name in crypto:
            if student_name not in attendance_recorded: 
                markAttendance(student_name, csv_file, "FINAL ATTENDANCE STATUS: ABSENT AS THE STUDENT DID NOT SPEND MINIMUM TIME IN CLASS")
                print("FINAL ATTENDANCE STATUS: ABSENT AS THE STUDENT DID NOT SPEND MINIMUM TIME IN CLASS :", student_name)
        # Check for students who never entered the class and mark them as totally absent
        totally_absent_students = all_class_names - set(attendance.keys())-set(crypto.keys())
        for student_name in totally_absent_students:
            markAttendance(student_name, csv_file, "FINAL ATTENDANCE STATUS: TOTALLY ABSENT")
            print("FINAL ATTENDANCE STATUS: TOTALLY ABSENT :", student_name)
        # Close the CSV file
        with open(csv_file, 'a') as f:
            f.close()
        
        break

# After attendance is marked, send the email with the CSV file
send_email(csv_file)

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

Encoding Complete
Alert: Student NAMRATA_152 has entered the class!
FINAL ATTENDANCE STATUS PRESENT: NAMRATA_152
Alert: Student NAMRATA_152 has left the class!
Alert: Student NAMRATA_152 has entered the class!
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : Sayantika_151
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : Poulami_L207
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : Subhra_154
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : Souhardo_157
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : Namrata_152
Email sent successfully to namrata2882m@gmail.com


In [19]:
import cv2
import numpy as np
import face_recognition
import os
import csv
from datetime import datetime, timedelta
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication


# Set the path to the directory containing subdirectories for each student
path = 'goku'
images = []
classNames = []

# Loop through all subdirectories (each representing a student)
for student_dir in os.listdir(path):
    student_path = os.path.join(path, student_dir)
    
    if os.path.isdir(student_path):
        # Loop through all image files within the student's directory
        for image_file in os.listdir(student_path):
            image_path = os.path.join(student_path, image_file)
            
            if os.path.isfile(image_path):
                # Load the image and append it to the 'images' list
                curImg = cv2.imread(image_path)
                
                if curImg is not None:
                    images.append(curImg)
                    classNames.append(student_dir)  # Use the student's directory name as the class name
                else:
                    print(f"Warning: Unable to load image {image_path}")

# Now, 'images' contains all the loaded images, and 'classNames' contains their corresponding student names.


# Function to find encodings of known faces
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img, model="cnn")[0]  # Use the CNN model for face recognition
        encodeList.append(encode)
    return encodeList



# Function to mark attendance
def markAttendance(name, csv_file, alert_message=None):
    if name != "Unknown":  # Only mark attendance if the name is not "Unknown"
        with open(csv_file, 'a', newline='') as f:
            now = datetime.now()
            date = now.strftime('%Y-%m-%d')
            day_of_week = now.strftime('%A')
            time_stamp = now.strftime('%H:%M:%S')
            
            if alert_message:
                row = [name, date, day_of_week, time_stamp, alert_message]
            else:
                row = [name, date, day_of_week, time_stamp, alert_message]

            writer = csv.writer(f)
            writer.writerow(row)




# Function to send an email with the CSV file as an attachment
def send_email(csv_file):
    # Email configuration
    sender_email = 'subhrabandyopadhyay15@gmail.com'  # Change to your email
    sender_password = 'lzks uedv fqco egir'  # Change to your email password
    recipient_email = 'namrata2882m@gmail.com'  # Change to recipient's email
    smtp_server = 'smtp.gmail.com'  # Change to your SMTP server
    smtp_port = 587  # Change to your SMTP server's port

    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = 'Attendance Report'

    # Attach the CSV file
    with open(csv_file, 'rb') as file:
        part = MIMEApplication(file.read(), Name=os.path.basename(csv_file))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(csv_file)}"'
        msg.attach(part)

    # Connect to the SMTP server and send the email
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, sender_password)
        text = msg.as_string()
        server.sendmail(sender_email, recipient_email, text)
        server.quit()
        print(f"Email sent successfully to {recipient_email}")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

# Initialize known face encodings
encodeListKnown = findEncodings(images)
print('Encoding Complete')

# Initialize webcam capture
cap = cv2.VideoCapture(0)

# Generate a unique CSV file name based on the current timestamp
csv_file = f'Attendance_Book_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'

# Define the header row
header = ['  NAME  ','  DATE  ',   '  DAY_OF_WEEK  ', '  TIME-STAMP  ', '  ATTENDANCE-STATUS  ']
# Write the header row to the CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)


# Initialize attendance record
attendance = {}
crypto={}



# Set the threshold time for student presence (adjust as needed)
presence_threshold = timedelta(seconds=10)
absence_threshold = timedelta(seconds=5)
attendance_recorded = set()  # Initialize the set to keep track of recorded attendance

# Set the tolerance for face recognition (adjust as needed, lower values are stricter)
tolerance = 0.5

# Create a set of all class names
all_class_names = set(classNames)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS, model="cnn")  # Use the CNN model for face detection
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    # Create a list to keep track of students present in the current frame
    students_present = []

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace, tolerance=tolerance)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            student_name = classNames[matchIndex].upper()
            confidence = 100 - (faceDis[matchIndex] * 100)  # Calculate confidence percentage
            students_present.append(student_name)

            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, f"{student_name} ({confidence:.2f}%)", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)


            if student_name not in attendance:
                alert_message = f"Alert: Student {student_name} has entered the class!"
                print(alert_message)
                #markAttendance(student_name, csv_file, alert_message)
                attendance[student_name] = datetime.now()
                crypto[student_name]= datetime.now()
                
            else:
                last_seen = attendance[student_name]
                current_time = datetime.now()
                time_difference = current_time - last_seen

                # Check if the student has been present for at least 10 seconds
                if time_difference >= presence_threshold:
                    if student_name not in attendance_recorded:
                        markAttendance(student_name, csv_file, "FINAL ATTENDANCE STATUS: PRESENT")
                        attendance_recorded.add(student_name)
                        
                        print("FINAL ATTENDANCE STATUS PRESENT:", student_name)
                        
        else:
            # Unknown person logic
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red rectangle
            cv2.putText(img, "Unknown", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

           

    # Check for students who were previously marked as present but are not in the current frame
    for student_name, last_seen in list(attendance.items()):
        if student_name not in students_present:
            current_time = datetime.now()
            time_difference = current_time - last_seen
            if time_difference > absence_threshold:
                alert_message = f"Alert: Student {student_name} has left the class!"
                print(alert_message)
                #markAttendance(student_name, csv_file, alert_message)
                
                
                crypto[student_name]= datetime.now()
                del attendance[student_name]
                
                


    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # Before exiting the program, add a final attendance entry for each student
        for student_name in crypto:
            if student_name not in attendance_recorded: 
                markAttendance(student_name, csv_file, "FINAL ATTENDANCE STATUS: ABSENT AS THE STUDENT DID NOT SPEND MINIMUM TIME IN CLASS")
                print("FINAL ATTENDANCE STATUS: ABSENT AS THE STUDENT DID NOT SPEND MINIMUM TIME IN CLASS :", student_name)
        # Check for students who never entered the class and mark them as totally absent
        totally_absent_students = all_class_names - set(attendance.keys())-set(crypto.keys())
        for student_name in totally_absent_students:
            markAttendance(student_name, csv_file, "FINAL ATTENDANCE STATUS: TOTALLY ABSENT")
            print("FINAL ATTENDANCE STATUS: TOTALLY ABSENT :", student_name)
        # Close the CSV file
        with open(csv_file, 'a') as f:
            f.close()
        
        break

# After attendance is marked, send the email with the CSV file
send_email(csv_file)

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

Encoding Complete
Alert: Student NAMRATA_152 has entered the class!
FINAL ATTENDANCE STATUS PRESENT: NAMRATA_152
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : SOUHARDO_157
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : SUBHRA_154
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : POULAMI_L207
FINAL ATTENDANCE STATUS: TOTALLY ABSENT : SAYANTIKA_151
Email sent successfully to namrata2882m@gmail.com
